In [1]:
#!pip install --user Fiona-1.8.6-cp37-cp37m-win_amd64.whl
#!pip install --user Shapely-1.6.4.post2-cp37-cp37m-win_amd64.whl
#!pip install --user GDAL-2.4.1-cp37-cp37m-win_amd64.whl

In [1]:
import xosrm
from libfunc import *
import pandas as pd
import os
from IPython.display import display

In [20]:
schedule_data = open_excel_file(
    file_path='C:\\Users\\andre\\OneDrive\\Рабочий стол\\sav\\iorder\\VN_iorder_Косарев.xlsx')
# Dict of coordinate for each day
full_dict = schedule(schedule_data)

In [3]:
''' 
Расчет пробега на каждый день недели для точек в заданом порядке
Расписание НЕ ДОЛЖНО содержать ТТ Новый для КПК
Вывод: словарь дистанций
'''

# Координаты точки отправления и прибытия (офиса)
source = [28.5673, 49.2226]
dest = [28.5673, 49.2226]

data = calc_dist(full_dict)
display(df_first(data))
display(df_second(data))

,1-monday,2-tuesday,3-wednesday,4-thursday,5-friday
even_part_of_coords,885.4109,905.9279,875.5211,958.5975,899.3688
odd_part_of_coords,887.3683,905.1207,870.8273,955.8857,882.6220


,1-monday,2-tuesday,3-wednesday,4-thursday,5-friday
even_part_of_coords,877.5063,893.4162,862.2244,963.2426,874.1797
odd_part_of_coords,877.1112,891.6605,864.8232,947.7129,865.0977


{'first_month_part': {'even_part_of_coords': {'1-monday': 885.4109,
   '2-tuesday': 905.9279,
   '3-wednesday': 875.5210999999999,
   '4-thursday': 958.5975,
   '5-friday': 899.3688000000001},
  'odd_part_of_coords': {'1-monday': 887.3683000000001,
   '2-tuesday': 905.1206999999999,
   '3-wednesday': 870.8273,
   '4-thursday': 955.8856999999999,
   '5-friday': 882.622}},
 'second_month_part': {'even_part_of_coords': {'1-monday': 877.5063,
   '2-tuesday': 893.4162,
   '3-wednesday': 862.2244000000001,
   '4-thursday': 963.2425999999999,
   '5-friday': 874.1796999999999},
  'odd_part_of_coords': {'1-monday': 877.1111999999999,
   '2-tuesday': 891.6605,
   '3-wednesday': 864.8231999999999,
   '4-thursday': 947.7129,
   '5-friday': 865.0976999999999}}}

In [21]:
''' 
Расчет TRIP_ROUTE (Жадный алгоритм)
асчет пробега на каждый день недели для точек с оптимальной последовательностью
Расписание ДОЛЖНО содержать Новый для КПК первой точкой на день с индесом 0 или 1
Вывод: словарь дистанций
'''    
# вызов функции
data = calc_trips(full_dict)
# Отображение таблиц
display(df_first(data))
display(df_second(data))


,1-monday,2-tuesday,3-wednesday,4-thursday,5-friday
even_part_of_coords,230.4240,0.0000,238.6798,0.0000,0.0000
odd_part_of_coords,231.2874,58.9348,232.0578,253.8673,239.5475


,1-monday,3-wednesday
even_part_of_coords,230.4240,238.6798
odd_part_of_coords,230.6847,232.0578


In [19]:
'''
Расчет оптимальных пробегов для всех расписаний в папке
На вход: путь к папке с файлами расписаний
Вывод: таблицы с пробегами по каждому дню
'''
# Путь к папке с расписаниями
path = 'C:\\Users\\andre\\OneDrive\\Рабочий стол\\sav\\iorder\\'

# Вызов функции
calc_folder(path)    

iorder  -  DP_iorder_Боженко
Первая половина месяца:


,1-monday,2-tuesday,3-wednesday,4-thursday,5-friday
even_part_of_coords,103.8868,0.0000,17.4726,88.2912,257.6436
odd_part_of_coords,117.7402,276.2076,0.0000,82.4906,277.1025


Вторая половина месяца:


,1-monday,2-tuesday,3-wednesday,4-thursday,5-friday
even_part_of_coords,103.8868,0.0000,17.4726,88.2912,64.1542
odd_part_of_coords,117.7402,276.2076,0.0000,82.4906,277.1025


iorder  -  LV_iorder_ТП_Еффі
Первая половина месяца:


...

,1-monday,2-tuesday,3-wednesday,4-thursday,5-friday
even_part_of_coords,0.0000,0.0000,0.0000,0.0000,0.0000
odd_part_of_coords,30.6173,494.9459,38.1171,214.7246,178.4035


Вторая половина месяца:


""
even_part_of_coords
odd_part_of_coords


iorder  -  VN_iorder_Косарев


HTTPError: HTTP Error 400: Bad Request

In [263]:
import folium
from folium import plugins
import os

# Create map object
m = folium.Map(location=[46.4289,30.6508], zoom_start=12)

plugins.ScrollZoomToggler().add_to(m)

plugins.Fullscreen(
position='topright',
title='FullScreen',
title_cancel='Exit me',
force_separate_button=True).add_to(m)

# Geojson data
overlay_route = os.path.join('data', 'route.json')
# overlay_trip = os.path.join('data', 'trip.json')

# Geojson overlay
folium.GeoJson(overlay_route, name='lumar-route').add_to(m)
# folium.GeoJson(overlay_trip, name='lumar-trip').add_to(m)

In [264]:
m # Отобразить карту